Introduction: Short-term asset return is a challenging quantity to predict. Efficient markets produce near-Normal daily returns with no significant correlation between $r_t$ , $r_{t−1}$. This exercise is a limited exercise in supervised learning. You are expected to explore multiple features or may use a set of features from Table 1 without an expectation of predictive powers. Original and final selected features should be sufficiently large.

# Objective
Your objective is to produce a model to predict positive moves (up trend) using machine learning models as specified in the below section. Your proposed solution should be comprehensive with the detailed feature engineering and model architecture.

- Choose one ticker of your interest from the index, equity, ETF, crypto token, or commodity.
- Predict trend only, for a short-term return (example: daily, 6 hours). Limit prediction to binomial classification: the dependent variable is best labeled [0, 1]. Avoid using [-1, 1] as class labels.
- Analysis should be comprehensive with detailed feature engineering, data pre-processing, model building, and evaluation.

Devise your own approach on how to categorise extremely small near-zero returns (drop from training sample, group with positive/negative). The threshold will strongly depend on your ticker. Example: small positive returns below 0.25% can be labelled as negative.

**Table 1: Features List**

| Feature        | Formula                                                                 | Description               |
|----------------|-------------------------------------------------------------------------|---------------------------|
| O-C, H-L       | Open - Close, High - Low                                                | intraday price range      |
| Sign           | sign [rt = ln \(\frac{P_t}{P_{t-1}}\)]                                  | sign of return or momentum|
| Past Returns   | \(r_{t-1}, r_{t-2}, \ldots\)                                            | lagged returns            |
| Momentum       | \(P_t - P_{t-k}\)                                                       | price change over k period|
| Moving Average | \(SMA_i = \frac{1}{n}\sum_{i=0}^{n-1} P_{t-i}\)                         | simple moving average     |
| Exponential MA | \(EMA_i = EMA_{i-1} + \alpha (P_t - EMA_{i-1})\), \(\alpha = \frac{2}{(N_{obs} + 1)}\) | recursive, \(\alpha = \frac{2}{(N_{obs} + 1)}\) |

Number of features to include is a design choice. There is no one recommended set of features for all assets. Length of dataset is another design choice. If predicting short-term return sign (for daily move), then training and testing over up to 5-year period should be sufficient. Making sense of instructions below is part of the task.

# Questions

1. What is the cost function of Logistic Regression? Explain in Detail.
2. What are voting classifiers in ensemble learning?
3. Follow the 7-steps to model building for your selected ticker,

    - produce a model to predict positive moves (up trend) using Support Vector Machine (SVM) model.
    - tune hyperparameters for the estimator and present the best model.
    - investigate the prediction quality using area under ROC curve, confusion matrix and classification report.

Note: Choice of kernels and number of hyperparameters to be optimized for the best model are design choices.